In [1]:
from langchain_groq import ChatGroq
from crewai import Agent
from crewai import Crew
from crewai import Task
from langchain.document_loaders import PyPDFLoader
from langchain_community.tools.tavily_search import TavilySearchResults

In [2]:
import os
GROQ_API_KEY = os.environ["GROQ_API_KEY"]
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [3]:
#model 
llm=ChatGroq(
        temperature = 0.1,
        model="llama3-70b-8192",
        api_key= os.getenv("GROQ_API_KEY"),
)

In [4]:
import requests

pdf_url = "https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf"
response = requests.get(pdf_url)

with open("attenstion_is_all_you_need.pdf", "wb") as file:
    file.write(response.content)

In [5]:
# Load the PDF
loader = PyPDFLoader("attenstion_is_all_you_need.pdf")
documents = loader.load()

In [6]:
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

In [7]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

embeddings = download_hugging_face_embeddings()

C:\Users\DELL\AppData\Local\Temp\ipykernel_13684\16761998.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
d:\anaconda\envs\agentrag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever()

In [13]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [16]:
response = rag_chain.invoke("What is the main contribution of the paper 'Attention Is All You Need'?",
)

In [18]:
response['result']

'The provided context does not explicitly mention the paper "Attention Is All You Need". However, based on the content, it appears to be describing a model that uses self-attention mechanisms, which is the main contribution of the paper "Attention Is All You Need" by Vaswani et al. (2017). The paper introduced the Transformer model, which relies solely on self-attention mechanisms to process input sequences, eliminating the need for traditional recurrent or convolutional neural networks.'

In [29]:
from dotenv import load_dotenv
load_dotenv()
TAVILY_API_KEY = os.environ["TAVILY_API_KEY"]
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
tavily_tool = TavilySearchResults(k=3)

In [30]:
reuslt=tavily_tool.run("What is the main contribution of the paper 'Attention Is All You Need'?"  )
print(reuslt[0]['content'])

"Attention Is All You Need"( is a 2017 landmark( paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer "Transformer (machine learning model)"), based on the attention mechanism proposed in 2014 by Bahdanau et al.( It is considered a foundational( paper in modern artificial intelligence, and a main contributor to the AI boom, as the transformer approach has become the main architecture of a wide [...] In 2017, the original (100M-sized) encoder-decoder transformer model was proposed in the "Attention is all you need" paper. At the time, the focus of the research was on improving seq2seq for machine translation, by removing its recurrence to process all tokens in parallel, but preserving its dot-product attention mechanism to keep its text processing performance.( This led to the introduction of a multi-head attention model that was easier to parallelize due to the use of independent heads a

In [32]:
from langchain_core.tools import tool
@tool
def router_tool(question):
  """Router Function"""
  if 'self-attention' in question:
    return 'vectorstore'
  else:
    return 'web_search'

In [33]:
#creste router agent
router_agent = Agent(
     role='Router',
    goal='Route user question to a vectorstore or web search',
    backstory=(
    "You are an expert at routing a user question to a vectorstore or web search."
    "Use the vectorstore for questions on concept related to Retrieval-Augmented Generation."
    "You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web-search."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [34]:
#Creste Retriever Agent
Retriever_Agent = Agent(
role="Retriever",
goal="Use the information retrieved from the vectorstore to answer the question",
backstory=(
    "You are an assistant for question-answering tasks."
    "Use the information present in the retrieved context to answer the question."
    "You have to provide a clear concise answer."
),
verbose=True,
allow_delegation=False,
llm=llm,
)

In [35]:
Grader_agent =  Agent(
  role='Answer Grader',
  goal='Filter out erroneous retrievals',
  backstory=(
    "You are a grader assessing relevance of a retrieved document to a user question."
    "If the document contains keywords related to the user question, grade it as relevant."
    "It does not need to be a stringent test.You have to make sure that the answer is relevant to the question."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [36]:
hallucination_grader = Agent(
    role="Hallucination Grader",
    goal="Filter out hallucination",
    backstory=(
        "You are a hallucination grader assessing whether an answer is grounded in / supported by a set of facts."
        "Make sure you meticulously review the answer and check if the response provided is in alignmnet with the question asked"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [37]:
answer_grader = Agent(
    role="Answer Grader",
    goal="Filter out hallucination from the answer.",
    backstory=(
        "You are a grader assessing whether an answer is useful to resolve a question."
        "Make sure you meticulously review the answer and check if it makes sense for the question asked"
        "If the answer is relevant generate a clear and concise response."
        "If the answer gnerated is not relevant then perform a websearch using 'web_search_tool'"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)